# Support Ticket Classification - Data Exploration

This notebook explores the support ticket dataset to understand the data distribution, patterns, and characteristics before building the classification models.

## Objectives
- Understand the dataset structure and quality
- Analyze ticket categories and priority distributions
- Explore text patterns and common themes
- Identify potential challenges for model training

Author: ML Engineering Team

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('default')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

## 1. Load and Initial Data Exploration

In [ ]:
# Load the dataset
df = pd.read_csv('../data/tickets.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nDataset Info:")
df.info()

In [ ]:
# Display first few rows
print("First 5 rows:")
df.head()

In [ ]:
# Check for missing values
print("Missing Values:")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Count': missing_values,
    'Percentage': missing_percentage
})
print(missing_df[missing_df['Count'] > 0])

## 2. Target Variables Analysis

In [ ]:
# Analyze Ticket Type distribution
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
ticket_type_counts = df['Ticket Type'].value_counts()
plt.pie(ticket_type_counts.values, labels=ticket_type_counts.index, autopct='%1.1f%%')
plt.title('Ticket Type Distribution')

plt.subplot(1, 2, 2)
ticket_type_counts.plot(kind='bar')
plt.title('Ticket Type Count')
plt.xlabel('Ticket Type')
plt.ylabel('Count')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("Ticket Type Statistics:")
print(f"Total categories: {len(ticket_type_counts)}")
print(f"Most common: {ticket_type_counts.index[0]} ({ticket_type_counts.iloc[0]} tickets)")
print(f"Least common: {ticket_type_counts.index[-1]} ({ticket_type_counts.iloc[-1]} tickets)")

In [ ]:
# Analyze Ticket Priority distribution
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
priority_counts = df['Ticket Priority'].value_counts()
plt.pie(priority_counts.values, labels=priority_counts.index, autopct='%1.1f%%')
plt.title('Ticket Priority Distribution')

plt.subplot(1, 2, 2)
priority_counts.plot(kind='bar')
plt.title('Ticket Priority Count')
plt.xlabel('Priority')
plt.ylabel('Count')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("Priority Statistics:")
print(f"Total priority levels: {len(priority_counts)}")
print(f"Balance ratio: {priority_counts.max() / priority_counts.min():.2f}")

In [ ]:
# Cross-analysis: Ticket Type vs Priority
plt.figure(figsize=(14, 8))
cross_tab = pd.crosstab(df['Ticket Type'], df['Ticket Priority'])
sns.heatmap(cross_tab, annot=True, fmt='d', cmap='Blues')
plt.title('Ticket Type vs Priority Heatmap')
plt.xlabel('Priority')
plt.ylabel('Ticket Type')
plt.tight_layout()
plt.show()

print("Cross-tabulation (Ticket Type vs Priority):")
print(cross_tab)

## 3. Text Analysis

In [ ]:
# Basic text statistics
df['text_length'] = df['Ticket Description'].str.len()
df['word_count'] = df['Ticket Description'].str.split().str.len()

print("Text Statistics:")
print(f"Average text length: {df['text_length'].mean():.2f} characters")
print(f"Median text length: {df['text_length'].median():.2f} characters")
print(f"Average word count: {df['word_count'].mean():.2f} words")
print(f"Median word count: {df['word_count'].median():.2f} words")

# Plot text length distribution
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(df['text_length'], bins=50, alpha=0.7)
plt.title('Text Length Distribution')
plt.xlabel('Character Count')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(df['word_count'], bins=50, alpha=0.7, color='orange')
plt.title('Word Count Distribution')
plt.xlabel('Word Count')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.scatter(df['text_length'], df['word_count'], alpha=0.5)
plt.title('Text Length vs Word Count')
plt.xlabel('Character Count')
plt.ylabel('Word Count')

plt.tight_layout()
plt.show()

In [ ]:
# Text length by ticket type
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
sns.boxplot(data=df, x='Ticket Type', y='text_length')
plt.title('Text Length by Ticket Type')
plt.xlabel('Ticket Type')
plt.ylabel('Character Count')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
sns.boxplot(data=df, x='Ticket Priority', y='text_length')
plt.title('Text Length by Priority')
plt.xlabel('Priority')
plt.ylabel('Character Count')

plt.tight_layout()
plt.show()

## 4. Word Cloud Analysis

In [ ]:
# Simple text cleaning function for word clouds
def clean_text_for_wc(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Clean all descriptions
df['cleaned_text'] = df['Ticket Description'].apply(clean_text_for_wc)

# Generate word cloud for all tickets
all_text = ' '.join(df['cleaned_text'])
wordcloud_all = WordCloud(width=800, height=400, background_color='white').generate(all_text)

plt.figure(figsize=(15, 10))
plt.subplot(2, 3, 1)
plt.imshow(wordcloud_all, interpolation='bilinear')
plt.title('All Tickets Word Cloud')
plt.axis('off')

# Word clouds by ticket type
ticket_types = df['Ticket Type'].unique()[:5]  # Limit to 5 types for visibility
for i, ticket_type in enumerate(ticket_types, 2):
    if i > 6:  # Limit subplot positions
        break
    
    type_text = ' '.join(df[df['Ticket Type'] == ticket_type]['cleaned_text'])
    wordcloud_type = WordCloud(width=400, height=300, background_color='white').generate(type_text)
    
    plt.subplot(2, 3, i)
    plt.imshow(wordcloud_type, interpolation='bilinear')
    plt.title(f'{ticket_type} Word Cloud')
    plt.axis('off')

plt.tight_layout()
plt.show()

## 5. Common Words and Phrases Analysis

In [ ]:
# Function to get most common words
def get_most_common_words(texts, n=20):
    all_words = []
    for text in texts:
        if pd.notna(text):
            words = str(text).lower().split()
            all_words.extend([word for word in words if len(word) > 2])  # Filter short words
    
    word_freq = Counter(all_words)
    return word_freq.most_common(n)

# Get most common words overall
common_words = get_most_common_words(df['cleaned_text'], 20)
print("Top 20 Most Common Words:")
for i, (word, count) in enumerate(common_words, 1):
    print(f"{i:2d}. {word:<15} ({count} occurrences)")

In [ ]:
# Most common words by ticket type
plt.figure(figsize=(18, 12))

for i, ticket_type in enumerate(df['Ticket Type'].unique(), 1):
    if i > 6:  # Limit to 6 subplots
        break
    
    type_texts = df[df['Ticket Type'] == ticket_type]['cleaned_text']
    common_words_type = get_most_common_words(type_texts, 10)
    
    words = [word for word, count in common_words_type]
    counts = [count for word, count in common_words_type]
    
    plt.subplot(2, 3, i)
    plt.barh(words, counts)
    plt.title(f'Top Words: {ticket_type}')
    plt.xlabel('Frequency')
    
    # Reverse y-axis to show highest frequency at top
    plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

## 6. Data Quality Assessment

In [ ]:
# Check for duplicate tickets
duplicate_descriptions = df['Ticket Description'].duplicated().sum()
print(f"Duplicate ticket descriptions: {duplicate_descriptions}")

# Check for very short or empty descriptions
very_short = (df['text_length'] < 10).sum()
empty_descriptions = (df['text_length'] == 0).sum()

print(f"Very short descriptions (<10 chars): {very_short}")
print(f"Empty descriptions: {empty_descriptions}")

# Check for potential data quality issues
print(f"\nData Quality Summary:")
print(f"- Total tickets: {len(df)}")
print(f"- Tickets with missing descriptions: {df['Ticket Description'].isnull().sum()}")
print(f"- Tickets with missing type: {df['Ticket Type'].isnull().sum()}")
print(f"- Tickets with missing priority: {df['Ticket Priority'].isnull().sum()}")
print(f"- Average description length: {df['text_length'].mean():.1f} characters")
print(f"- Class balance (Type): {df['Ticket Type'].value_counts().max() / df['Ticket Type'].value_counts().min():.2f}")
print(f"- Class balance (Priority): {df['Ticket Priority'].value_counts().max() / df['Ticket Priority'].value_counts().min():.2f}")

## 7. Insights and Recommendations

In [ ]:
# Summary statistics for model planning
print("="*60)
print("DATA EXPLORATION SUMMARY")
print("="*60)

print(f"\n📊 DATASET OVERVIEW:")
print(f"  Total samples: {len(df):,}")
print(f"  Features: {len(df.columns)}")
print(f"  Target variables: Ticket Type ({len(df['Ticket Type'].unique())} classes), Ticket Priority ({len(df['Ticket Priority'].unique())} classes)")

print(f"\n🎯 CLASSIFICATION CHALLENGES:")
type_balance = df['Ticket Type'].value_counts().max() / df['Ticket Type'].value_counts().min()
priority_balance = df['Ticket Priority'].value_counts().max() / df['Ticket Priority'].value_counts().min()

print(f"  Ticket Type balance ratio: {type_balance:.2f} {'(Well balanced)' if type_balance < 2 else '(Imbalanced)'}")
print(f"  Priority balance ratio: {priority_balance:.2f} {'(Well balanced)' if priority_balance < 2 else '(Imbalanced)'}")

print(f"\n📝 TEXT CHARACTERISTICS:")
print(f"  Average text length: {df['text_length'].mean():.0f} characters")
print(f"  Average word count: {df['word_count'].mean():.0f} words")
print(f"  Text length range: {df['text_length'].min()} - {df['text_length'].max()} characters")

print(f"\n🔧 MODELING RECOMMENDATIONS:")
print(f"  1. Use TF-IDF vectorization with n-grams (1,2) for better context")
print(f"  2. Consider class weighting for priority prediction due to imbalance")
print(f"  3. Text preprocessing should include stopword removal and lemmatization")
print(f"  4. Maximum features around 5000 should capture important patterns")
print(f"  5. Use stratified sampling to maintain class distribution in train/test splits")

print(f"\n⚠️  DATA QUALITY CONCERNS:")
if duplicate_descriptions > 0:
    print(f"  - {duplicate_descriptions} duplicate descriptions found")
if very_short > 0:
    print(f"  - {very_short} very short descriptions may need special handling")
if df['Ticket Description'].isnull().sum() > 0:
    print(f"  - {df['Ticket Description'].isnull().sum()} missing descriptions")

print(f"\n✅ DATA STRENGTHS:")
print(f"  - Large dataset size suitable for deep learning")
print(f"  - Reasonable class balance for both targets")
print(f"  - Rich text content with diverse vocabulary")
print(f"  - Clear business problem with well-defined categories")

## 8. Next Steps

Based on this exploration, the next steps for building the classification models are:

1. **Data Preprocessing**: Implement comprehensive text cleaning
2. **Feature Engineering**: Create TF-IDF features with appropriate parameters
3. **Model Selection**: Start with Logistic Regression and compare with other algorithms
4. **Evaluation**: Use appropriate metrics considering class balance
5. **Validation**: Implement cross-validation and hold-out testing
6. **Deployment**: Save models and create prediction pipelines

The dataset appears suitable for building high-quality classification models with proper preprocessing and feature engineering.